In [ ]:
import copy

import torch
from torchsummary import summary

import numpy as np
import pandas as pd
import random

In [ ]:
from models import ResNet50

In [ ]:
from utils3 import evaluate, get_data_batch

In [ ]:
# parameters#############################
iid = 0 # if the data is IID or 0 not IID      
#########################################
########################################
unbalanced = 1# in non i.i.d. setting split the data between clients equally or not
########################################



num_users = 50# number of client
frac = 0.1 # fraction of the clients to be used for federated updates
n_epochs = 20
gpu = 0
optimizer = "sgd" #sgd or adam
local_batch_size =128 # batch size of local updates in each user
lr = 1 # learning rate
local_epochs = 10
loss_function = "CrossEntropyLoss" #//crosssssssssssss

percentage = 0  # percentage of strugglers

mu = 0.05  # proximal term constant



####################################################
num_groups = 2  # 0 for BatchNorm, > 0 for GroupNorm
if num_groups == 2:
    normalization_type = "BatchNorm"
else:
    normalization_type = "GroupNorm"

####################################################

In [ ]:
if iid:
    from iid import get_dataset, average_weights, weighted_average_weights, exp_details
else:
    from niid import get_dataset, average_weights, weighted_average_weights, exp_details

In [ ]:
exp_details("ResNet50", optimizer, lr, normalization_type, n_epochs, iid, frac,
            local_batch_size, local_epochs, unbalanced, num_users)


Experimental details:
    Model     : ResNet50
    Optimizer : sgd
    Learning  : 1
    Normalization  : BatchNorm
    Global Rounds   : 20

    Federated parameters:
    Non-IID - unbalanced
    NUmber of users  : 50
    Fraction of users  : 0.1
    Local Batch size   : 128
    Local Epochs       : 10



In [ ]:
train_dataset, test_dataset, user_groups = get_dataset(iid=iid, unbalanced=unbalanced,
                                                       num_users=num_users)

In [ ]:
train_dataset

In [ ]:
!pip install fedlab

In [ ]:
from tqdm import trange
from fedlab.utils.serialization import SerializationTool
from fedlab.utils.functional import get_best_gpu
from os import listdir

In [ ]:
from torch.nn import CrossEntropyLoss

from copy import deepcopy

from math import ceil

In [ ]:
from utils3 import get_args, get_model

In [ ]:
model = ResNet50(n_type=normalization_type)
# model = CNNCifar()

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
    device = torch.device("cpu")
    gpu = 0
else:
    print('CUDA is available!  Training on GPU ...')
    device = torch.device("cuda")
    gpu = 1

global_model = model.to(device)

# set the model to train
global_model.train()

In [ ]:
c_global = [
    torch.zeros_like(param, device=device)
    for param in global_model.parameters()
    if param.requires_grad
]

In [ ]:
# for REPRODUCIBILITY https://pytorch.org/docs/stable/notes/randomness.html
torch.manual_seed(2)

g = torch.Generator()
g.manual_seed(2)

np.random.seed(2)

In [ ]:
# copy weights
global_weights = model.state_dict()

In [ ]:
criterion = CrossEntropyLoss()

In [ ]:
client_num_in_total=len(user_groups)

In [ ]:
client_indices = range(client_num_in_total)

In [ ]:
comms_round=n_epochs
client_num_per_round=10
test_round=1

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
#train_dataset, test_dataset
trainloader = DataLoader(train_dataset,local_batch_size, shuffle=True)
testloader = DataLoader(test_dataset, local_batch_size, shuffle=True)

In [ ]:
from scaffold import SCAFFOLDTrainer

In [ ]:

client_list = [
    SCAFFOLDTrainer(
        client_id=client_id,
        global_model=global_model,
        trainloader=trainloader,
        valloader=testloader,
        batch_size=local_batch_size,
        lr=lr,
        criterion=criterion,
        epochs=local_epochs,
        cuda=train_on_gpu,
    )
    for client_id in range(client_num_in_total)
]

In [ ]:
acc=[]
lss=[]

In [ ]:
for r in range(comms_round):
    # select clients
    selected_clients = random.sample(client_indices, client_num_per_round)
    print(
        "\033[1;34mselected clients in round [{}]: {}\033[0m".format(
            r, selected_clients
        )
    )
    global_model_param = SerializationTool.serialize_model(global_model)
    c_delta_buffer = []
    y_delta_buffer = []
    # train
    for client_id in selected_clients:
        y_delta, c_delta = client_list[client_id].train(
            global_model_param, c_global
        )
        c_delta_buffer.append(c_delta)
        y_delta_buffer.append(y_delta)
    
    with torch.no_grad():
        # update global model
        for y_del in y_delta_buffer:
            for param, diff in zip(global_model.parameters(), y_del):
                param.data.add_(
                    diff.data * lr / client_num_per_round
                )
        # update global_c
        for c_delta in c_delta_buffer:
            for c_g, c_d in zip(c_global, c_delta):
                c_g.data.add_(c_d.data / client_num_in_total)

    if (r%1==0):

      print(r)
      avg_loss_g = 0  # global model loss
      avg_acc_g = 0  # global model accuracy
      avg_loss_l = 0  # localized model loss
      avg_acc_l = 0  # localized model accuracy
      
      for client_id in selected_clients:
        stats = client_list[client_id].eval(global_model_param, c_global)
        avg_loss_g += stats[0]
        avg_acc_g += stats[1]
        avg_loss_l += stats[2]
        avg_acc_l += stats[3]
        # display experiment results
      avg_loss_g /= client_num_per_round * test_round
      avg_acc_g /= client_num_per_round * test_round
      avg_loss_l /= client_num_per_round * test_round
      avg_acc_l /= client_num_per_round * test_round
      acc.append("\033[1;33m Global SCAFFOLD loss: {:.4f}\033[0m".format(avg_loss_g))
      lss.append("\033[1;33m Global SCAFFOLD accuracy: {:.2f}%\033[0m".format(avg_acc_g))
      print("\033[1;32m---------------------- result behind----------------------\033[0m")
      print("\033[1;33m Global SCAFFOLD loss: {:.4f}\033[0m".format(avg_loss_g))
      print("\033[1;33m Global SCAFFOLD accuracy: {:.2f}%\033[0m".format(avg_acc_g))
      print("\033[1;36m Localized SCAFFOLD loss: {:.4f}\033[0m".format(avg_loss_l))
      print("\033[1;36m Localized SCAFFOLD accuracy: {:.2f}%\033[0m".format(avg_acc_l))
# evaluate
avg_loss_g = 0  # global model loss
avg_acc_g = 0  # global model accuracy
avg_loss_l = 0  # localized model loss
avg_acc_l = 0  # localized model accuracy
for r in range(test_round):
    selected_clients = random.sample(client_indices,client_num_per_round)
    print(
        "\033[1;34mselected clients in round [{}]: {}\033[0m".format(
            r, selected_clients
        )
    )
    global_model_param = SerializationTool.serialize_model(global_model)
    for client_id in selected_clients:
        stats = client_list[client_id].eval(global_model_param, c_global)
        avg_loss_g += stats[0]
        avg_acc_g += stats[1]
        avg_loss_l += stats[2]
        avg_acc_l += stats[3]

# display experiment results
avg_loss_g /= client_num_per_round * test_round
avg_acc_g /= client_num_per_round * test_round
avg_loss_l /= client_num_per_round * test_round
avg_acc_l /= client_num_per_round * test_round
print("\033[1;32m---------------------- result behind----------------------\033[0m")
print("\033[1;33m Global SCAFFOLD loss: {:.4f}\033[0m".format(avg_loss_g))
print("\033[1;33m Global SCAFFOLD accuracy: {:.2f}%\033[0m".format(avg_acc_g))
print("\033[1;36m Localized SCAFFOLD loss: {:.4f}\033[0m".format(avg_loss_l))
print("\033[1;36m Localized SCAFFOLD accuracy: {:.2f}%\033[0m".format(avg_acc_l))

In [ ]:
acc


In [ ]:
lss

In [ ]:

# save train loss and accuracy
import pandas as pd


data = list(zip(lss, acc))
pd.DataFrame(data, columns=['train_loss','train_accuracy']).to_csv("scaffoldoutput.csv")


In [ ]:
from google.colab import files
files.download('scaffoldoutput.csv')